In [1]:
from spektral.data import Dataset
from spektral.data.graph import Graph
from spektral.data import DisjointLoader
from spektral.layers import ECCConv, GlobalSumPool
from spektral.data.loaders import BatchLoader
from spektral.models.general_gnn import GeneralGNN
from spektral.utils import tic, toc
from spektral.layers.convolutional import gcn_conv
from spektral.layers.convolutional import arma_conv

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.losses import MeanSquaredLogarithmicError
from tensorflow.keras.callbacks import EarlyStopping

import os
import logging
import numpy as np
import math

from numpy.random import rand
from numpy.random import randint
from random import randrange

import matplotlib.pyplot as plt
import networkx as nx

# suppress tensorflow log messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL
logging.getLogger('tensorflow').setLevel(logging.FATAL)


In [2]:
X = 100
Y = 100
N_GRAPHS = 1


In [3]:
G = nx.grid_2d_graph(X, Y)
inp = rand(G.number_of_nodes(), 1)
out = inp / 10

a = nx.to_numpy_array(G, dtype=np.float32)

print(a.shape)

nx.set_node_attributes(G, out, 'value')


(10000, 10000)


In [5]:
class Features(Dataset):
    """
    A dataset for the features test.
    """

    def __init__(self, nodes, feats, graphs, adjacency, **kwargs):
        self.nodes = nodes
        self.feats = feats
        self.graphs = graphs
        self.a = adjacency
        self.mask_tr = self.mask_va = self.mask_te = None

        self.download()

        super().__init__(**kwargs)

    def download(self):
        data = ...  # Download from somewhere

        # Create the directory
        if not os.path.exists(self.path):
            os.mkdir(self.path)

        # Write the data to file
        for i in range(self.graphs):
            x = rand(G.number_of_nodes(), self.feats).astype(np.float32)
            a = self.a
            y = x / 10
            e = np.full((G.number_of_nodes(), G.number_of_nodes(),
                        1), 10.0, dtype=np.float32)

            filename = os.path.join(self.path, f'graph_{i}')
            np.savez(filename, x=x, a=a, e=e, y=y)

        mask = np.zeros(self.nodes, dtype=np.bool8)

        self.mask_tr = mask
        self.mask_tr[0:int(self.nodes // (5/4))] = True
        self.mask_va = mask
        self.mask_va[int((self.nodes // (5/4)) + 1)
                         :int(self.nodes // (10/9))] = True
        self.mask_te = mask
        self.mask_te[int((self.nodes // (10/9)) + 1):self.nodes] = True

    def read(self):
        # We must return a list of Graph objects
        output = []

        for i in range(self.graphs):
            data = np.load(os.path.join(self.path, f'graph_{i}.npz'))
            output.append(
                Graph(x=data['x'], a=data['a'], y=data['y'], e=data['e'])
            )

        return output


In [6]:

ds = Features((X * Y), 1, N_GRAPHS, a)
graph = ds[0]
x, a, y = graph.x, graph.a, graph.y

mask_tr, mask_va, mask_te = ds.mask_tr, ds.mask_va, ds.mask_te


In [7]:
ds[0]

Graph(n_nodes=10000, n_node_features=1, n_edge_features=1, n_labels=1)

In [8]:
class ThisGCN(tf.keras.Model):
    """
    This model, with its default hyperparameters, implements the architecture
    from the paper:
    > [Semi-Supervised Classification with Graph Convolutional Networks](https://arxiv.org/abs/1609.02907)<br>
    > Thomas N. Kipf and Max Welling
    **Mode**: single, disjoint, mixed, batch.
    **Input**
    - Node features of shape `([batch], n_nodes, n_node_features)`
    - Weighted adjacency matrix of shape `([batch], n_nodes, n_nodes)`
    **Output**
    - Softmax predictions with shape `([batch], n_nodes, n_labels)`.
    **Arguments**
    - `n_labels`: number of channels in output;
    - `channels`: number of channels in first GCNConv layer;
    - `activation`: activation of the first GCNConv layer;
    - `output_activation`: activation of the second GCNConv layer;
    - `use_bias`: whether to add a learnable bias to the two GCNConv layers;
    - `dropout_rate`: `rate` used in `Dropout` layers;
    - `l2_reg`: l2 regularization strength;
    - `**kwargs`: passed to `Model.__init__`.
    """

    def __init__(
        self,
        n_labels,
        channels=16,
        activation="relu",
        output_activation="relu",
        use_bias=False,
        dropout_rate=0.5,
        l2_reg=2.5e-4,
        **kwargs,
    ):
        super().__init__(**kwargs)

        self.n_labels = n_labels
        self.channels = channels
        self.activation = activation
        self.output_activation = output_activation
        self.use_bias = use_bias
        self.dropout_rate = dropout_rate
        self.l2_reg = l2_reg
        reg = tf.keras.regularizers.l2(l2_reg)
        self._d0 = tf.keras.layers.Dropout(dropout_rate)
        self._arma0 = arma_conv.ARMAConv(
            channels, activation=activation, kernel_regularizer=reg, use_bias=use_bias
        )
        self._d1 = tf.keras.layers.Dropout(dropout_rate)
        self._arma1 = arma_conv.ARMAConv(
            n_labels, activation=output_activation, use_bias=use_bias
        )
        self.norm = tf.keras.layers.Normalization()

    def get_config(self):
        return dict(
            n_labels=self.n_labels,
            channels=self.channels,
            activation=self.activation,
            output_activation=self.output_activation,
            use_bias=self.use_bias,
            dropout_rate=self.dropout_rate,
            l2_reg=self.l2_reg,
        )

    def call(self, inputs):
        if len(inputs) == 2:
            x, a = inputs
        else:
            x, a, _ = inputs  # So that the model can be used with DisjointLoader

        #x = self._d0(x)
        #x = self._arma0([x, a])
        #x = self._d1(x)
        x = self._arma1([x, a])
        #x = self.norm(x)
        return x


In [9]:
model = ThisGCN(n_labels=ds.n_labels, output_activation='relu',
                use_bias=False, channels=8, activation='relu')
model.compile('adam', 'mean_squarred_error')
optimizer = Adam(lr=1e-3, beta_1=0.9, beta_2=0.999)
loss_fn = MeanSquaredError()


/home/martin/anaconda3/envs/spektral/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [10]:
# Training step
@tf.function
def train():
    with tf.GradientTape() as tape:
        predictions = model([x, a], training=True)
        # tf.keras.backend.eval(predictions)
        loss = loss_fn(y, predictions)
        loss += sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


In [59]:
EPOCHS = 50

# train()  # Warm up to ignore tracing times when timing
# tic()
for epoch in range(1, EPOCHS + 1):
    loss = train()
    # print(f'{epoch}/{EPOCHS + 1}')
print(f"Final loss = {loss}")


Final loss = 5.400334543992358e-07


In [60]:
pred = model([ds[0].x, ds[0].a], training=False)


In [61]:
print(f'inp \t\t target \t prediction')
for i in range(0, min(ds[0].n_nodes, 20)):
    print(f'{ds[0].x[i]} \t {ds[0].y[i]} \t {pred[i]}')


inp 		 target 	 prediction
[0.09152373] 	 [0.00915237] 	 [0.00964277]
[0.9299479] 	 [0.09299479] 	 [0.09159435]
[0.9526359] 	 [0.09526359] 	 [0.09397903]
[0.27284595] 	 [0.0272846] 	 [0.0273614]
[0.22789101] 	 [0.0227891] 	 [0.02308998]
[0.9432238] 	 [0.09432238] 	 [0.09316468]
[0.782502] 	 [0.0782502] 	 [0.07733753]
[0.07967036] 	 [0.00796704] 	 [0.00913838]
[0.88714087] 	 [0.08871409] 	 [0.08699435]
[0.02020095] 	 [0.0020201] 	 [0.00329766]
[0.7813558] 	 [0.07813558] 	 [0.07664625]
[0.40511513] 	 [0.04051151] 	 [0.04082731]
[0.93637955] 	 [0.09363796] 	 [0.09264733]
[0.88402045] 	 [0.08840205] 	 [0.0874404]
[0.7902052] 	 [0.07902052] 	 [0.07859133]
[0.9621381] 	 [0.09621381] 	 [0.09507436]
[0.5246358] 	 [0.05246358] 	 [0.05228867]
[0.6705537] 	 [0.06705537] 	 [0.06624425]
[0.3199243] 	 [0.03199243] 	 [0.03210551]
[0.7687994] 	 [0.07687994] 	 [0.07596046]
